<a href="https://colab.research.google.com/github/timsigit/oss-saki-ss19-exercises/blob/master/NER_Flair.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
! pip install flair

     |████████████████████████████████| 143kB 3.6MB/s 
     |████████████████████████████████| 798kB 45.7MB/s 
     |████████████████████████████████| 655kB 41.4MB/s 
     |████████████████████████████████| 133kB 50.6MB/s 
     |████████████████████████████████| 1.0MB 47.2MB/s 
  Stored in directory: /root/.cache/pip/wheels/bd/57/d3/907c3ee02d35e66f674ad0106e61f06eeeb98f6ee66a6cc3fe
  Stored in directory: /root/.cache/pip/wheels/c0/47/fb/8a64f89aecfe0059830479308ad42d62e898a3e3cefdf6ba28
  Stored in directory: /root/.cache/pip/wheels/15/ee/a8/6112173f1386d33eebedb3f73429cfa41a4c3084556bcee254
Successfully built sqlitedict mpld3 regex segtok


In [0]:
# imports 
from flair.datasets import Corpus
from flair.data_fetcher import NLPTaskDataFetcher

# file structure
columns = {0: 'text', 1: 'ner'}

# folder where training and test data are stored
data_folder = '/content/gdrive/My Drive'

# 1.0 is full data
downsample = 1.0

# train file name
train_file = 'train_res_bilou_sentences.txt'

# test file name
test_file = 'test_res_bilou_sentences.txt'

# get the corpus
corpus: Corpus = NLPTaskDataFetcher.load_column_corpus(data_folder, columns,
                                                             train_file=train_file,
                                                             test_file=test_file,
                                                             dev_file=None).downsample(downsample)
print(corpus)

# make the tag dictionary from the corpus
tag_dictionary = corpus.make_tag_dictionary(tag_type='ner')
print(tag_dictionary.idx2item)

2019-06-17 06:21:26,552 Reading data from /content/gdrive/My Drive
2019-06-17 06:21:26,558 Train: /content/gdrive/My Drive/train_res_bilou_sentences.txt
2019-06-17 06:21:26,560 Dev: None
2019-06-17 06:21:26,564 Test: /content/gdrive/My Drive/test_res_bilou_sentences.txt


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: DeprecationWarning: Call to deprecated function (or staticmethod) load_column_corpus. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
/usr/local/lib/python3.6/dist-packages/flair/data_fetcher.py:312: DeprecationWarning: Call to deprecated function (or staticmethod) read_column_data. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  train_file, column_format
/usr/local/lib/python3.6/dist-packages/flair/data_fetcher.py:318: DeprecationWarning: Call to deprecated function (or staticmethod) read_column_data. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  test_file, column_format


Corpus: 10713 train + 1190 dev + 4399 test sentences
[b'<unk>', b'O', b'"B-Companies', b'"L-Companies', b'B-Designation', b'I-Designation', b'L-Designation', b'"I-Companies', b'B-Degree', b'I-Degree', b'L-Degree', b'U-Designation', b'U-Degree', b'-', b'"U-Companies', b'ner', b'<START>', b'<STOP>']


In [0]:
# initialize embeddings. Experiment with different embedding types to see what gets the best results
from flair.embeddings import TokenEmbeddings, WordEmbeddings, StackedEmbeddings,FlairEmbeddings
from typing import List, Union, Dict

# used embeddings
embedding_types: List[TokenEmbeddings] = [
    WordEmbeddings('glove'),
    FlairEmbeddings('news-forward'),
    FlairEmbeddings('news-backward'),
]


embeddings: StackedEmbeddings = StackedEmbeddings(embeddings=embedding_types)

# initialize sequence tagger
from flair.models import SequenceTagger

tagger: SequenceTagger = SequenceTagger(hidden_size=256,
                                        embeddings=embeddings,
                                        tag_dictionary=tag_dictionary,
                                        tag_type='ner',
                                        use_crf=True)



2019-06-17 06:22:08,987 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/embeddings-v0.4.1/big-news-forward--h2048-l1-d0.05-lr30-0.25-20/news-forward-0.4.1.pt not found in cache, downloading to /tmp/tmp2yqa5j1k


100%|██████████| 73034624/73034624 [00:10<00:00, 6954937.66B/s] 

2019-06-17 06:22:20,012 copying /tmp/tmp2yqa5j1k to cache at /root/.flair/embeddings/news-forward-0.4.1.pt


2019-06-17 06:22:20,096 removing temp file /tmp/tmp2yqa5j1k
2019-06-17 06:22:27,634 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/embeddings-v0.4.1/big-news-backward--h2048-l1-d0.05-lr30-0.25-20/news-backward-0.4.1.pt not found in cache, downloading to /tmp/tmp9lzi_ks8


100%|██████████| 73034575/73034575 [00:04<00:00, 17056648.90B/s]

2019-06-17 06:22:32,472 copying /tmp/tmp9lzi_ks8 to cache at /root/.flair/embeddings/news-backward-0.4.1.pt


2019-06-17 06:22:32,552 removing temp file /tmp/tmp9lzi_ks8


In [7]:
# initialize trainer
from flair.trainers import ModelTrainer

trainer: ModelTrainer = ModelTrainer(tagger, corpus)

model_name = 'resources/taggers/resume-ner-2f_100_ts'

# actual training of the model
trainer.train(model_name,
              learning_rate=0.1,
              mini_batch_size=32,
              #anneal_with_restarts=True,
              max_epochs=150)

2019-06-17 06:23:14,017 ----------------------------------------------------------------------------------------------------
2019-06-17 06:23:14,020 Evaluation method: MICRO_F1_SCORE
2019-06-17 06:23:14,321 ----------------------------------------------------------------------------------------------------
2019-06-17 06:23:15,209 epoch 1 - iter 0/335 - loss 45.43640137
2019-06-17 06:23:35,661 epoch 1 - iter 33/335 - loss 7.17236656
2019-06-17 06:23:56,687 epoch 1 - iter 66/335 - loss 5.69326781
2019-06-17 06:24:20,536 epoch 1 - iter 99/335 - loss 4.98563991
2019-06-17 06:24:41,198 epoch 1 - iter 132/335 - loss 4.50315747
2019-06-17 06:25:07,118 epoch 1 - iter 165/335 - loss 4.21615085
2019-06-17 06:25:25,762 epoch 1 - iter 198/335 - loss 3.91208668
2019-06-17 06:25:49,072 epoch 1 - iter 231/335 - loss 3.71061208
2019-06-17 06:26:12,382 epoch 1 - iter 264/335 - loss 3.54476160
2019-06-17 06:26:35,961 epoch 1 - iter 297/335 - loss 3.37963618
2019-06-17 06:26:57,396 epoch 1 - iter 330/335

{'dev_loss_history': [tensor(1.9916, device='cuda:0'),
  tensor(1.2113, device='cuda:0'),
  tensor(0.9337, device='cuda:0'),
  tensor(0.8506, device='cuda:0'),
  tensor(0.8018, device='cuda:0'),
  tensor(0.7568, device='cuda:0'),
  tensor(0.7505, device='cuda:0'),
  tensor(0.6977, device='cuda:0'),
  tensor(0.6990, device='cuda:0'),
  tensor(0.6974, device='cuda:0'),
  tensor(0.6289, device='cuda:0'),
  tensor(0.6829, device='cuda:0'),
  tensor(0.6283, device='cuda:0'),
  tensor(0.6189, device='cuda:0'),
  tensor(0.5982, device='cuda:0'),
  tensor(0.6103, device='cuda:0'),
  tensor(0.6469, device='cuda:0'),
  tensor(0.6261, device='cuda:0'),
  tensor(0.5791, device='cuda:0'),
  tensor(0.5865, device='cuda:0'),
  tensor(0.6024, device='cuda:0'),
  tensor(0.5930, device='cuda:0'),
  tensor(0.5677, device='cuda:0'),
  tensor(0.6166, device='cuda:0'),
  tensor(0.5939, device='cuda:0'),
  tensor(0.5684, device='cuda:0'),
  tensor(0.5928, device='cuda:0'),
  tensor(0.5890, device='cuda:0'),


In [0]:
## Bash commands can be run by prefixing the command with ‘!’.
!ls "/content/drive/My Drive/"

# New Section